In [1]:
#import packages needed
import xarray as xr
import dask
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import scipy
import os
import glob as glob
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import calendar

In [2]:
#decompose advection terms into ubar u' dtdxbar dtxdx'

#do wrt to monthly mean and wrt to annual mean


In [3]:
#functions that get each heat budget variable calculated.
#general form so can use any dataset
def diff_T(T):
    dTdt = T.differentiate(coord = 'time',datetime_unit= 's')
    dTdx = T.differentiate('lon') / (110e3 * np.cos(T.lat * np.pi / 180))
    dTdy = T.differentiate('lat') / (110e3 )
    dTdz = (T[:,0] - T[:,-1])/float(T.level[-1])
    return dTdt[:,:-1], dTdx[:,:-1], dTdy[:,:-1], dTdz

def advection(u, v, w, dx, dy, dz):
    uadv = u*dx
    vadv = v*dy
    wadv = w*dz
    return uadv, vadv, wadv

def get_weights(INP):
    weights = INP.level
    #add a 0m surface layer
    wt=np.array([0.])
    for i in range(len(weights.values)):
        NW = 2*weights.values[i]-wt[i]
        wt = np.insert(wt,i+1,NW)
    thickness = wt[1:]-wt[:-1]
    thickness_DA = xr.DataArray(thickness, coords={'level': INP.level},
                 dims=['level'])
    return thickness_DA

def weighted_avg(inp, weights):
    avg=inp.weighted(weights).mean('level')
    return avg


def get_clim(dict_name):
    dict1={}
    #unpack dict, get climatology, repack
    for key in dict_name.keys():
        var = dict_name[key]
        var = var.groupby('time.month').mean('time')
        dict1[key] = var
    return dict1


In [7]:
#now do the same for ec-earth3
T = xr.open_mfdataset('/home/z5113258/Documents/data/CMIP6/EC-Earth3/lev_int/THETAO/*.nc').thetao
U = xr.open_mfdataset('/home/z5113258/Documents/data/CMIP6/EC-Earth3/lev_int/UO/*.nc').uo
V = xr.open_mfdataset('/home/z5113258/Documents/data/CMIP6/EC-Earth3/lev_int/VO/*.nc').vo
W = xr.open_mfdataset('/home/z5113258/Documents/data/CMIP6/EC-Earth3/lev_int/WO/*.nc').wo
Q = xr.open_mfdataset('/home/z5113258/Documents/data/CMIP6/EC-Earth3/EC-Earth3_HFDS.nc').hfds
SW = xr.open_mfdataset('/home/z5113258/Documents/data/CMIP6/EC-Earth3/RSNTDS/*.nc').rsntds
#get only to 50m
T = T[:,:5]
U = U[:,:5]
V = V[:,:5]
W = W[:,4]
#now put lat lon of uvw to T
U = U.assign_coords({'lat': T.lat, 'lon': T.lon})
V = V.assign_coords({'lat': T.lat, 'lon': T.lon})
W = W.assign_coords({'lat': T.lat, 'lon': T.lon})
#rename lev coord to level
T = T.rename({'lev':'level'})
U = U.rename({'lev':'level'})
V = V.rename({'lev':'level'})
W = W.rename({'lev':'level'})

In [8]:
dTdtg, dTdxg, dTdyg, dTdzg = diff_T(T)

In [9]:
uadvg, vadvg, wadvg = advection(U, V, W, dTdxg, dTdyg, dTdzg)

In [10]:
gqnet = (Q-SW*((0.67*np.exp(-50))+((1-0.67)*np.exp(-50/17))))/(3986*1026*50)

In [11]:
#calcualate weights of levels
weights = get_weights(T)
#now get wtd avg
dTdt_gw = weighted_avg(dTdtg, weights)
uwg = weighted_avg(uadvg, weights)
vwg = weighted_avg(vadvg, weights)
wwg = wadvg

In [12]:
#alse get weighted avergae of u cur, v cur, and dtdx and dtdy
#decompose advection terms to see where the bias is coming from - temp or current
#GODAS
dTdxgw = weighted_avg(dTdxg, weights)
dTdygw = weighted_avg(dTdyg, weights)

gUw = weighted_avg(U, weights)
gVw = weighted_avg(V, weights)

In [13]:
#now calcaulte cliamotlogy of terms
dT_c = dTdt_gw.groupby('time.month').mean('time')
u_c = uwg.groupby('time.month').mean('time')
v_c = vwg.groupby('time.month').mean('time')
w_c = wwg.groupby('time.month').mean('time')
q_c = gqnet.groupby('time.month').mean('time')

In [14]:
dxgC = dTdxgw.groupby('time.month').mean('time')
dygC = dTdygw.groupby('time.month').mean('time')
dzgC = dTdzg.groupby('time.month').mean('time')
ugC = gUw.groupby('time.month').mean('time')
vgC = gVw.groupby('time.month').mean('time')
wgC = W.groupby('time.month').mean('time')

In [15]:
#now save data
#now convert to netcdf
dT_c.to_netcdf('/home/z5113258/Documents/data/CMIP6/EC-Earth3/climatology/EC_dtdt.nc')
u_c.to_netcdf('/home/z5113258/Documents/data/CMIP6/EC-Earth3/climatology/EC_uadv.nc')
v_c.to_netcdf('/home/z5113258/Documents/data/CMIP6/EC-Earth3/climatology/EC_vadv.nc')
w_c.to_netcdf('/home/z5113258/Documents/data/CMIP6/EC-Earth3/climatology/EC_wadv.nc')
q_c.to_netcdf('/home/z5113258/Documents/data/CMIP6/EC-Earth3/climatology/EC_qnet.nc')
dxgC.to_netcdf('/home/z5113258/Documents/data/CMIP6/EC-Earth3/climatology/EC_dtdx.nc')
dygC.to_netcdf('/home/z5113258/Documents/data/CMIP6/EC-Earth3/climatology/EC_dtdy.nc')
dzgC.to_netcdf('/home/z5113258/Documents/data/CMIP6/EC-Earth3/climatology/EC_dtdz.nc')
ugC.to_netcdf('/home/z5113258/Documents/data/CMIP6/EC-Earth3/climatology/EC_ucur.nc')
vgC.to_netcdf('/home/z5113258/Documents/data/CMIP6/EC-Earth3/climatology/EC_vcur.nc')
wgC.to_netcdf('/home/z5113258/Documents/data/CMIP6/EC-Earth3/climatology/EC_wcur.nc')



/home/z5113258/anaconda3/lib/python3.8/site-packages/dask/array/numpy_compat.py:41: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/home/z5113258/anaconda3/lib/python3.8/site-packages/dask/array/numpy_compat.py:41: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/home/z5113258/anaconda3/lib/python3.8/site-packages/dask/array/numpy_compat.py:41: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/home/z5113258/anaconda3/lib/python3.8/site-packages/dask/array/numpy_compat.py:41: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/home/z5113258/anaconda3/lib/python3.8/site-packages/dask/array/numpy_compat.py:41: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/home/z5113258/anaconda3/lib/python3.8/site-packages/dask/array/numpy_compat.py:41: RuntimeWarning: invalid value encountered in true_divide
  x = np.